In [1]:
import pandas as pd
import numpy as np
from scipy import sparse as sps
import nltk
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
import lenskit
import lenskit.crossfold as xf
from  lenskit.crossfold import TTPair

In [2]:
saved = open("pickle/game_reviews.pickle","rb")
game_reviews = pickle.load(saved)
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [3]:
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [4]:
#game_reviews['review'] = game_reviews['review'].applymap(str)
#game_reviews['review'] = game_reviews['review'].astype(str)

In [49]:
type(reviews['user'][0])

str

In [55]:
user_game = open("pickle/user_games.pickle","rb")
user_games = pickle.load(user_game)
user_games.head()

,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [57]:
#user_games_list = user_games[['item_id', 'user_id']]
user_games_list = user_games.rename(columns={'user_id': 'user', 'item_id': 'item'})
user_games_list.head()

,user,item,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [13]:
reviews = game_reviews[['item_id', 'user_id','review']]
reviews = reviews.rename(columns={'user_id': 'user', 'item_id': 'item'})
reviews.shape

(58430, 3)

In [70]:
reviews.tail(15)

,item,user,review
58415,413150,76561198289386531,I realy like this game it has gotten me to und...
58416,242760,GodLoveGuru,"Good Game Play, encountered a problem with cli..."
58417,427730,sexyawp,dont ask
58418,280790,76561198296402247,"Great game, specially if you need to past time..."
58419,570,JustMielThings,Good one
58420,261030,76561198306599751,I cried in the end its so sadding ]'; I wish l...
58421,730,Ghoustik,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...
58422,570,76561198310819422,Well Done
58423,233270,76561198312638244,this is a very fun and nice 80s themed shooter...
58424,130,76561198312638244,if you liked Half life i would really recommen...


In [42]:
A = set(user_games_list['user'])

In [40]:
B = set(reviews['user'])

In [63]:
len(B.difference(A))

2845

In [76]:
user_games_list.loc[user_games_list['item'] == 570]

,user,item,item_name


In [75]:
reviews.loc[reviews['item'] == 570]

,item,user,review
40,570,DJKamBer,"Awesome graphics, texture. Reqiures HARDCORE t..."
68,570,76561198070565427,"w,"
71,570,76561198076909484,GOD GAME
88,570,76561198075277292,1K mmr players be like:Phantom Assasin has dis...
166,570,YoHuNz,fsdgfdsgsdfgfd
238,570,L1J1N,Great way to learn Russian!♥♥♥♥
282,570,GGargula,problemas com serve e conexão
288,570,76561198097333029,Sold my soul to this game would do it again 10/10
323,570,2xDelorean,Played for a bit...enjoyed it. ( ͡° ͜ʖ ͡°)
359,570,benedictcuminherbatch,Its okay I guess


### join user-item and reviws

In [1]:
result = pd.merge(user_games_list, reviews, how = 'outer', on=['item', 'user'])
result.shape

NameError: name 'pd' is not defined

In [74]:
result.loc[result['item'] == 570]

,user,item,item_name,review


In [59]:
result

,user,item,item_name,review
0,76561197970982479,10,Counter-Strike,NaN
1,76561197970982479,20,Team Fortress Classic,NaN
2,76561197970982479,30,Day of Defeat,NaN
3,76561197970982479,40,Deathmatch Classic,NaN
4,76561197970982479,50,Half-Life: Opposing Force,NaN
5,76561197970982479,60,Ricochet,NaN
6,76561197970982479,70,Half-Life,NaN
7,76561197970982479,130,Half-Life: Blue Shift,NaN
8,76561197970982479,300,Day of Defeat: Source,NaN
9,76561197970982479,240,Counter-Strike: Source,NaN


### item count and remove user less than (n)

In [34]:
def groupby_count(df, group, count):
    game_count = pd.DataFrame()
    game_count['count'] = df.groupby(group)[count].count()
    return game_count

In [35]:
def locate_user(df, condition):
    user_n = df.loc[df['count'] < condition ]
    return user_n

In [36]:
game_count = groupby_count(reviews, 'user', 'item')

## removing less than 5 reviews provider

In [37]:
user_5 = locate_user(game_count, 5)

In [38]:
user_less_5 = user_5.index
user_less_5

Index(['--000--', '--ace--', '--ionex--', '-Azsael-', '-Beave-', '-GM-Dragon',
       '-I_AM_EPIC-', '-Kenny', '-Mad-', '-PRoSlayeR-',
       ...
       'zucchin1', 'zudikas', 'zukuta', 'zumpo', 'zunbae', 'zvanik',
       'zwanzigdrei', 'zy0705', 'zynxgameth', 'zzoptimuszz'],
      dtype='object', name='user', length=22185)

In [40]:
game_count.shape

(25457, 1)

In [41]:
pruned_data_5 = reviews.set_index('user').drop(user_less_5)
pruned_data_5.reset_index(inplace = True)
pruned_data_5.head()

,user,item,review
0,evcentric,248820,A suitably punishing roguelike platformer. Wi...
1,evcentric,370360,"""Run for fun? What the hell kind of fun is that?"""
2,evcentric,237930,"Elegant integration of gameplay, story, world ..."
3,evcentric,263360,"Random drops and random quests, with stat poin..."
4,evcentric,107200,Fun balance of tactics and strategy. Potentia...


In [42]:
reviews.nunique()

item       3682
user      25457
review    55312
dtype: int64

In [43]:
pruned_data_5.nunique()

user       3272
item       2708
review    21520
dtype: int64

## check user item

In [44]:
pruned_data_5.loc[pruned_data_5['user'] == 'kenkaniff']

,user,item,review
18433,kenkaniff,42680,This is honestly the worst Call of Duty ever. ...
18434,kenkaniff,242050,Great game! Recommend it!
18435,kenkaniff,209160,Not that great of a game actually! The multipl...
18436,kenkaniff,8190,This is one of my favourite games! It is reall...
18437,kenkaniff,24240,"Love it, can't stop playing it!!"


In [45]:
pruned_data_5.iloc[133]

user      76561198088944293
item                 437701
review      were is my ♥♥♥♥
Name: 133, dtype: object

## Partition by user

In [46]:
from lenskit.batch import MultiEval
from lenskit.crossfold import partition_users, SampleN, partition_rows
from lenskit import topn
from tf_idf import tf_idf
from lenskit import batch, topn, util

In [19]:
pairs = list(partition_rows(pruned_data_5, 5))
truth = pd.concat((p.test for p in pairs), ignore_index=True)
truth.to_csv(r'results/steam/truth.csv')

## Evaluation

In [47]:
def algo_eval(path, algo, dataset):
    evaluation = batch.MultiEval(path=path, predict=False, recommend=True)
    evaluation.add_algorithms(algos=algo)
    evaluation.add_datasets(data=dataset)
    evaluation.run()

In [48]:
def ndcg(file_name, truth):
    recs = pd.read_parquet(file_name)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    ndcg = rla.compute(recs, truth)
    return ndcg

## fit

In [20]:
%%time
tf_idf_eval = batch.MultiEval(path='results/steam/time', predict=False, recommend=True)
tf_idf_eval.add_algorithms(algos=[tf_idf()],anme = 'tf-idf')
tf_idf_eval.add_datasets(data=pairs)
tf_idf_eval.run()

/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_d

CPU times: user 15min 21s, sys: 24.8 s, total: 15min 46s
Wall time: 15min 20s


/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


## ndcg

In [32]:
ndcg_tfidf_pop = ndcg('results/steam/time/recommendations.parquet', truth)
ndcg_tfidf_pop.head()

/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


ndcg
user         RunId          
-2SV-vuLB-Kg 1      0.041484
             2      0.000000
             3      0.311048
             5      0.050007
-SEVEN-      2      0.054555

## compare

In [25]:
from lenskit.algorithms import als, basic, item_knn, user_knn
from lenskit.metrics import topn as tnmetrics
from lenskit.algorithms.basic import Fallback
from lenskit.algorithms.als import BiasedMF, ImplicitMF
from lenskit.algorithms.implicit import BPR

In [27]:
algo_ii = item_knn.ItemItem(20, center=False, aggregate='sum')
#algo_uu = user_knn.UserUser(30, center=False, aggregate='sum')
algo_pop = basic.Popular()
algo_mf = ImplicitMF(40)
algo_bpr = BPR()

In [28]:
%%time
eval = MultiEval(path='results/compare/time', predict=False, recommend=True)
eval.add_datasets(pairs, name='steam')
#eval.add_algorithms(basic.Popular(),name='Pop')
eval.add_algorithms(algos=[algo_ii, algo_pop,algo_mf,algo_bpr], parallel=True)
eval.run()

/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_d

CPU times: user 13min 55s, sys: 21.4 s, total: 14min 16s
Wall time: 4min 31s


In [3]:
ndcg_algo_pop = ndcg('results/compare/time/recommendations.parquet', truth)

NameError: name 'truth' is not defined

In [56]:
legend = pd.read_csv("results/compare/time/runs.csv")
legend = legend.set_index('RunId').loc[:,'AlgoStr']
legend

RunId
1                 ItemItem(nnbrs=20, msize=None)
2     als.ImplicitMF(features=40, reg=0.1, w=40)
3                 ItemItem(nnbrs=20, msize=None)
4     als.ImplicitMF(features=40, reg=0.1, w=40)
5                 ItemItem(nnbrs=20, msize=None)
6     als.ImplicitMF(features=40, reg=0.1, w=40)
7                 ItemItem(nnbrs=20, msize=None)
8     als.ImplicitMF(features=40, reg=0.1, w=40)
9                 ItemItem(nnbrs=20, msize=None)
10    als.ImplicitMF(features=40, reg=0.1, w=40)
Name: AlgoStr, dtype: object

In [57]:
ndcg_algo = ndcg_algo.join(legend, on='RunId')

In [58]:
ndcg_algo.head(15)

ndcg  \
user                    RunId             
-2SV-vuLB-Kg            1      0.030712   
                        2      0.109971   
                        5      0.044042   
-SEVEN-                 1      0.025380   
                        2      0.033049   
                        3      0.058356   
                        5      0.142531   
-_PussyDestroyer_-      1      0.079238   
                        4      0.044287   
                        5      0.086064   
00000000000000000001227 2      0.130537   
                        3      0.073389   
                        5      0.078346   
01189958889189157253    1      0.163092   
                        3      0.028652   

                                                                  AlgoStr  
user                    RunId                                              
-2SV-vuLB-Kg            1                  ItemItem(nnbrs=20, msize=None)  
                        2      als.ImplicitMF(features=40, reg=0.1, w=40)  
                        5                  ItemItem(nnbrs=20, msize=None)  
-SEVEN-                 1                  ItemItem(nnbrs=20, msize=None)  
                        2      als.ImplicitMF(features=40, reg=0.1, w=40)  
                        3                  ItemItem(nnbrs=20, msize=None)  
                        5                  ItemItem(nnbrs=20, msize=None)  
-_PussyDestroyer_-      1                  ItemItem(nnbrs=20, msize=None)  
                        4      als.ImplicitMF(features=40, reg=0.1, w=40)  
                        5                  ItemItem(nnbrs=20, msize=None)  
00000000000000000001227 2      als.ImplicitMF(features=40, reg=0.1, w=40)  
                        3                  ItemItem(nnbrs=20, msize=None)  
                        5                  ItemItem(nnbrs=20, msize=None)  
01189958889189157253    1                  ItemItem(nnbrs=20, msize=None)  
                        3                  ItemItem(nnbrs=20, msize=None)

## Partition_user

In [26]:
pairs_user = list(partition_users(pruned_data_5, 5, xf.SampleN(5)))
truth = pd.concat((p.test for p in pairs_user), ignore_index=True)
#truth.to_csv(r'results/steam/truth_user.csv')

In [24]:
eval = MultiEval('my-eval', predict=False, recommend=True)
eval.add_datasets(pairs_user, name='steam')
eval.add_algorithms(tf_idf(), name='tf_idf')
eval.run()

/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/fastparquet/util.py:221: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  inferred_dtype = infer_dtype(column)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_d

In [25]:
ndcg_user = ndcg('my-eval/recommendations.parquet', truth)

/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [26]:
ndcg_user

,,ndcg
user,RunId,
-2SV-vuLB-Kg,3,0.122735
-SEVEN-,1,0.217110
-_PussyDestroyer_-,5,0.168011
00000000000000000001227,4,0.254280
01189958889189157253,5,0.343112
04061993,4,0.205600
08254669696969696969,4,0.185234
091263,4,0.442134
09879655452567,3,0.316315


## Test

In [131]:
for i, tp in enumerate(xf.partition_users(pruned_data, 5, xf.SampleN(3))):
    tp.train.to_csv('steam/train_pruned-%d.csv' % (i,))
    tp.train.to_parquet('steam/train_pruned-%d.parquet' % (i,))
    tp.test.to_csv('steam/test_pruned-%d.csv' % (i,))
    tp.test.to_parquet('steam/test_pruned-%d.parquet' % (i,))

/home/amifaraj/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [50]:
tt_tuples =[]
for i, tp in enumerate(xf.partition_users(pruned_data_5, 5, xf.SampleN(3))):
    tuple = (tp.train, tp.test)
    tt_tuples.append(tuple)

In [132]:
test_data=[]
tt_tuples =[]
for i in range(0,3):
    train = pd.read_csv('steam/train_pruned-{}.csv'.format(i))
    test = pd.read_csv('steam/test_pruned-{}.csv'.format(i))
    test_data.append(test)
    tuple = (train, test)
    tt_tuples.append(tuple)

In [19]:
eval = MultiEval('my-eval', recommend=10)
eval.add_datasets(pairs, name='steam')
eval.add_algorithms(tf_idf(), name='tf_idf')
eval.run()

KeyError: "['rating'] not in index"

In [133]:
eval = MultiEval('my-eval', recommend=10)
eval.add_datasets(tt_tuples, name='steam')
eval.add_algorithms(tf_idf(), name='tf_idf')
eval.run()

TypeError: sequence item 132: expected str instance, float found